In [204]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [205]:
df=pd.read_parquet(r'E:\Data Science\PF\ML_streamlit\Datos\ML_1.parquet')

In [206]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,categories,fecha,cantidad,analisis_sentimiento,stars_y,useful
0,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Latin American,2020-03-01,1,2.0,5.0,2.0
1,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Fast Food,2020-03-01,1,2.0,5.0,2.0
2,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Latin American,2020-06-01,1,1.5,3.5,2.0
3,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Fast Food,2020-06-01,1,1.5,3.5,2.0
4,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Latin American,2020-10-01,1,2.0,5.0,3.0


In [207]:
df['name'].nunique()

5798

In [208]:
df['categories'].value_counts()

categories
American          137181
Mexican            26023
Fast Food          20722
East Asian         13974
Latin American      6487
African             1807
Name: count, dtype: int64

In [209]:
df['latitude'].value_counts()

latitude
27.812603    308
27.921942    288
27.976757    225
34.418889    216
27.777410    210
            ... 
39.945703      1
39.956166      1
34.439762      1
39.959695      1
28.064665      1
Name: count, Length: 7985, dtype: int64

Creamos los dummies

In [210]:
df_dummies=pd.get_dummies(df['categories'], dtype='int')

In [211]:
df_dummies = pd.concat([df_dummies, df], axis=1)

In [212]:
df_dummies.drop(['categories','business_id','name','address','postal_code','is_open','state','fecha'], axis=1, inplace=True)

In [213]:
df_dummies.head()

,African,American,East Asian,Fast Food,Latin American,Mexican,city,latitude,longitude,stars_x,review_count,cantidad,analisis_sentimiento,stars_y,useful
0,0,0,0,0,1,0,Elfers,28.217288,-82.733344,4.5,24,1,2.0,5.0,2.0
1,0,0,0,1,0,0,Elfers,28.217288,-82.733344,4.5,24,1,2.0,5.0,2.0
2,0,0,0,0,1,0,Elfers,28.217288,-82.733344,4.5,24,1,1.5,3.5,2.0
3,0,0,0,1,0,0,Elfers,28.217288,-82.733344,4.5,24,1,1.5,3.5,2.0
4,0,0,0,0,1,0,Elfers,28.217288,-82.733344,4.5,24,1,2.0,5.0,3.0


In [214]:
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206194 entries, 0 to 206193
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   African               206194 non-null  int32  
 1   American              206194 non-null  int32  
 2   East Asian            206194 non-null  int32  
 3   Fast Food             206194 non-null  int32  
 4   Latin American        206194 non-null  int32  
 5   Mexican               206194 non-null  int32  
 6   city                  206194 non-null  object 
 7   latitude              206194 non-null  float64
 8   longitude             206194 non-null  float64
 9   stars_x               206194 non-null  float32
 10  review_count          206194 non-null  int64  
 11  cantidad              206194 non-null  int64  
 12  analisis_sentimiento  206194 non-null  float64
 13  stars_y               206194 non-null  float64
 14  useful                206194 non-null  float64
dtype

In [215]:
y=df_dummies['city']
X=df_dummies.drop(['city'], axis=1)


In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [217]:
from sklearn.preprocessing import LabelEncoder

# Crear instancia de LabelEncoder
encoder = LabelEncoder()

# Ajustar y transformar 'y' para convertir las etiquetas de ciudad a valores numéricos
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [218]:
from sklearn.ensemble import RandomForestClassifier

# Crear el modelo de clasificación
modelo_rf = RandomForestClassifier(random_state=42)

# Entrenar el modelo con datos codificados
modelo_rf.fit(X_train, y_train_encoded)

# Realizar predicciones
predicciones = modelo_rf.predict(X_test)

In [219]:
# Decodificar las predicciones para obtener los nombres de las ciudades
predicciones_ciudades = encoder.inverse_transform(predicciones)

In [220]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Asumiendo que `y_test` son tus etiquetas verdaderas y `predictions` las predicciones del modelo
print(classification_report(y_test_encoded, predicciones))
print("Accuracy:", accuracy_score(y_test, predicciones))
# Para AUC-ROC, asegúrate de que es un problema de clasificación binaria o calcula el AUC para cada clase individualmente en problemas multiclase
# print("AUC-ROC:", roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr'))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99        76
           1       0.99      1.00      0.99        87
           2       1.00      1.00      1.00       124
           3       0.99      1.00      0.99       285
           4       0.98      1.00      0.99        91
           5       1.00      1.00      1.00        11
           6       1.00      1.00      1.00       433
           8       1.00      1.00      1.00       194
           9       1.00      0.66      0.79        35
          10       0.94      0.83      0.88        18
          11       1.00      0.98      0.99        44
          12       1.00      1.00      1.00        15
          13       1.00      1.00      1.00       105
          14       0.96      0.98      0.97       153
          15       1.00      0.99      0.99        69
          16       1.00      1.00      1.00       157
          17       1.00      1.00      1.00       102
          18       1.00    

c:\Users\juan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

In [221]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define el modelo
model = RandomForestClassifier()

# Define el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Configura la búsqueda
search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Ejecuta la búsqueda
search.fit(X_train, y_train_encoded)

# Mejores parámetros y puntuación
print("Mejores parámetros:", search.best_params_)
print("Mejor puntuación:", search.best_score_)

c:\Users\juan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score

#Evaluamos el rendimiento
train_accuracy=accuracy_score(y_train_encoded, search.predict(X_train))
test_accuracy=accuracy_score(y_test_encoded, search.predict(X_test))

print('Train accuracy:', train_accuracy)
print('Test accuracy:', test_accuracy)

if train_accuracy > test_accuracy + 0.05:  # Umbral de 0.05 es arbitrario
    print("Posible sobreajuste detectado.")
else:
    print("No se detecta sobreajuste significativo.")

In [ ]:
from joblib import dump

# Guarda el mejor modelo
mejor_modelo = search.best_estimator_
dump(mejor_modelo, 'mejor_modelo.joblib')